# Decision Analysis Project 2

- Mateusz Tabaszewski 151945
- Bartłomiej Pukacki 151942

This is our notebook dedicated to the second project from decision analysis classes, in this notebook we set out to implement our own versions of UTA and AHP methods used for multi-criteria decision analysis (MCDA). 

Sections present in this notebook:
* Imports - imports used for the notebook
* Utilities - general functions and classes which can be used outside the implemented methods
* Data - showcase of the constructed dataset
* UTA - implementation of UTA method
* AHP - implementation of AHP method
* Use - comparison of methods including PROMETHEE II results

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)

## Utilities

In [17]:
def get_pairwise_comparisons(data, column, boundaries, cost=False):
    """
    Returns pairwise comparisons based on specified boundaries
    
    Parameters:
        - data: dataframe with data
        - column: name of the column for which the comparison will be performed
        - boundaries: list of 8 boundaries of differences for which the Saaty's scale preference intensity increases
            e.g. with list [1,2,3,4,5,6,10,12] Saaty's preference of 9 applied for difference in values >12,
            the preference of 8 is applied for differences in interval (10, 12], 1 for [0, 1] etc.
        - cost: set to true to compare values of cost type criteria. Default = False
    """
    proportions = []
    # Apply comparison based on boundaries
    for i in range(len(data)):
        row = []
        for j in range(len(data)):
            if data.at[j, column] != 0:
                diff = data.at[i, column] - data.at[j, column]
                if cost:
                    diff *= -1
                    
                if diff < 0:
                    row.append("L")
                elif diff <= boundaries[0]:
                    row.append(1)
                elif diff <= boundaries[1]:
                    row.append(2)
                elif diff <= boundaries[2]:
                    row.append(3)
                elif diff <= boundaries[3]:
                    row.append(4)
                elif diff <= boundaries[4]:
                    row.append(5)
                elif diff <= boundaries[5]:
                    row.append(6)
                elif diff <= boundaries[6]:
                    row.append(7)
                elif diff <= boundaries[7]:
                    row.append(8)
                else:
                    row.append(9)

            else:
                row.append(None)
        proportions.append(row)

    # Create inverse values
    for i in range(len(data)):
        for j in range(len(data)):
            if proportions[i][j] == "L":
                proportions[i][j] = round(1/proportions[j][i], 2)

    df_proportions = pd.DataFrame(proportions, columns=data['id'], index=data['id'])
    return df_proportions

## Data


id | name                          | price | critic_score | user_score | length | genres   | num_of_achievements |
---|-------------------------------|-------|--------------|------------|--------|----------|---------------------|
0  | Dark Souls: Remastered        | 150   | 84           | 83         | 44     | 11       | 41                  |
1  | Dark Souls III                | 200   | 89           | 90         | 49     | 11       | 43                  |
2  | Terraria                      | 46    | 81           | 81         | 102    | 9        | 115                 |
3  | Baldur's Gate 3               | 250   | 96           | 89         | 107    | 8        | 54                  |
4  | Dave the Diver                | 92    | 90           | 83         | 32     | 7        | 43                  |
5  | Rust                          | 153   | 69           | 65         | 37     | 10       | 92                  |
6  | Hollow Knight                 | 68    | 90           | 91         | 42     | 10       | 63                  |
7  | Portal 2                      | 46    | 95           | 89         | 14     | 7        | 51                  |
8  | Vampire Survivors             | 20    | 86           | 83         | 25     | 9        | 204                 |
9  | Hades                         | 115   | 93           | 88         | 49     | 9        | 49                  |
10 | Subnautica                    | 139   | 87           | 86         | 43     | 5        | 17                  |
11 | Dishonored                    | 45    | 88           | 83         | 18     | 8        | 80                  |
12 | Ori and the Will of the Wisps | 108   | 90           | 89         | 16     | 10       | 37                  |
13 | Inside                        | 72    | 93           | 83         | 4      | 6        | 14                  |
14 | The Forest                    | 72    | 83           | 75         | 28     | 9        | 45                  |
15 | Skyrim                        | 90    | 96           | 86         | 114    | 11       | 75                  |
16 | Teardown                      | 120   | 80           | 81         | 22     | 6        | 27                  |
17 | Dying Light                   | 90    | 74           | 81         | 36     | 5        | 78                  |
18 | Enter the Gungeon             | 68    | 82           | 80         | 62     | 4        | 54                  |
19 | Payday 3                      | 169   | 66           | 31         | 10     | 5        | 22                  |
20 | Kao the Kangaroo              | 129   | 65           | 75         | 8      | 8        | 26                  |
21 | Assassin's Creed Unity        | 120   | 72           | 56         | 35     | 11       | 57                  |
22 | Trials Fusion                 | 80    | 79           | 71         | 23     | 2        | 51                  |
23 | The Sims 3                    | 28    | 83           | 78         | 78     | 3        | 65                  |
24 | Titan Souls                   | 68    | 74           | 61         | 4      | 8        | 27                  |

In [4]:
data = pd.read_csv("data.csv")
data

,id,name,price,critic_score,user_score,length,genres,num_of_achievements
0,0,Dark Souls: Remastered,150,84,83,44,11,41
1,1,Dark Souls III,200,89,90,49,11,43
2,2,Terraria,46,81,81,102,9,115
3,3,Baldur's Gate 3,250,96,89,107,8,54
4,4,Dave the Diver,92,90,83,32,7,43
5,5,Rust,153,69,65,37,10,92
6,6,Hollow Knight,68,90,91,42,10,63
7,7,Portal 2,46,95,89,14,7,51
8,8,Vampire Survivors,20,86,83,25,9,204
9,9,Hades,115,93,88,49,9,49


## UTA

## AHP

### Criteria hierarchy:

Goal: buying a video game that is described by quality and quantity measures

```tree
goal
├── quality
│   ├── user_score
│   ├── critic_score
│   └── genres
└── quantity
    ├── price
    ├── length
    └── num_of_achievements
```

### Pairwise comparisons of hierarchy elements:

<br>
User score difference preference decision boundaries: [2,4,5,6,7,9,10,15]

| US |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|    1 | 5    | 1    | 6    | 1    | 5    | 9    | 1    | 1    | 5    | 1    | 2    | 5    | 1    | 5    | 8    | 2    | 6    | 6    | 7    |    9 | 8    | 9    | 9    | 8    | 9    |
|    2 | 1    | 0.17 | 1    | 0.17 | 1    | 9    | 0.14 | 0.17 | 1    | 0.2  | 0.33 | 1    | 0.17 | 1    | 4    | 0.33 | 1    | 1    | 1    |    9 | 4    | 9    | 7    | 2    | 9    |
|    3 | 4    | 1    | 6    | 1    | 4    | 9    | 1    | 1    | 4    | 1    | 2    | 4    | 1    | 4    | 8    | 2    | 6    | 6    | 6    |    9 | 8    | 9    | 9    | 8    | 9    |
|    4 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|    5 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 1    | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.14 | 0.11 | 0.11 | 0.11 | 0.12 |    9 | 0.14 | 6    | 0.25 | 0.12 | 2    |
|    6 | 6    | 1    | 7    | 1    | 6    | 9    | 1    | 1    | 6    | 2    | 3    | 6    | 1    | 6    | 9    | 3    | 7    | 7    | 8    |    9 | 9    | 9    | 9    | 8    | 9    |
|    7 | 4    | 1    | 6    | 1    | 4    | 9    | 1    | 1    | 4    | 1    | 2    | 4    | 1    | 4    | 8    | 2    | 6    | 6    | 6    |    9 | 8    | 9    | 9    | 8    | 9    |
|    8 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|    9 | 3    | 1    | 5    | 1    | 3    | 9    | 0.5  | 1    | 3    | 1    | 1    | 3    | 1    | 3    | 8    | 1    | 5    | 5    | 6    |    9 | 8    | 9    | 9    | 7    | 9    |
|   10 | 2    | 0.5  | 3    | 0.5  | 2    | 9    | 0.33 | 0.5  | 2    | 1    | 1    | 2    | 0.5  | 2    | 8    | 1    | 3    | 3    | 4    |    9 | 8    | 9    | 8    | 6    | 9    |
|   11 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|   12 | 4    | 1    | 6    | 1    | 4    | 9    | 1    | 1    | 4    | 1    | 2    | 4    | 1    | 4    | 8    | 2    | 6    | 6    | 6    |    9 | 8    | 9    | 9    | 8    | 9    |
|   13 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|   14 | 0.17 | 0.12 | 0.25 | 0.12 | 0.17 | 7    | 0.11 | 0.12 | 0.17 | 0.12 | 0.12 | 0.17 | 0.12 | 0.17 | 1    | 0.12 | 0.25 | 0.25 | 0.33 |    9 | 1    | 9    | 2    | 0.5  | 8    |
|   15 | 2    | 0.5  | 3    | 0.5  | 2    | 9    | 0.33 | 0.5  | 2    | 1    | 1    | 2    | 0.5  | 2    | 8    | 1    | 3    | 3    | 4    |    9 | 8    | 9    | 8    | 6    | 9    |
|   16 | 1    | 0.17 | 1    | 0.17 | 1    | 9    | 0.14 | 0.17 | 1    | 0.2  | 0.33 | 1    | 0.17 | 1    | 4    | 0.33 | 1    | 1    | 1    |    9 | 4    | 9    | 7    | 2    | 9    |
|   17 | 1    | 0.17 | 1    | 0.17 | 1    | 9    | 0.14 | 0.17 | 1    | 0.2  | 0.33 | 1    | 0.17 | 1    | 4    | 0.33 | 1    | 1    | 1    |    9 | 4    | 9    | 7    | 2    | 9    |
|   18 | 0.5  | 0.14 | 1    | 0.17 | 0.5  | 8    | 0.12 | 0.17 | 0.5  | 0.17 | 0.25 | 0.5  | 0.17 | 0.5  | 3    | 0.25 | 1    | 1    | 1    |    9 | 3    | 9    | 6    | 1    | 9    |
|   19 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 |    1 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 |
|   20 | 0.17 | 0.12 | 0.25 | 0.12 | 0.17 | 7    | 0.11 | 0.12 | 0.17 | 0.12 | 0.12 | 0.17 | 0.12 | 0.17 | 1    | 0.12 | 0.25 | 0.25 | 0.33 |    9 | 1    | 9    | 2    | 0.5  | 8    |
|   21 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.17 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 |    9 | 0.11 | 1    | 0.12 | 0.11 | 0.33 |
|   22 | 0.12 | 0.11 | 0.14 | 0.11 | 0.12 | 4    | 0.11 | 0.11 | 0.12 | 0.11 | 0.12 | 0.12 | 0.11 | 0.12 | 0.5  | 0.12 | 0.14 | 0.14 | 0.17 |    9 | 0.5  | 8    | 1    | 0.2  | 7    |
|   23 | 0.33 | 0.12 | 0.5  | 0.12 | 0.33 | 8    | 0.12 | 0.12 | 0.33 | 0.14 | 0.17 | 0.33 | 0.12 | 0.33 | 2    | 0.17 | 0.5  | 0.5  | 1    |    9 | 2    | 9    | 5    | 1    | 9    |
|   24 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.5  | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.12 | 0.11 | 0.11 | 0.11 | 0.11 |    9 | 0.12 | 3    | 0.14 | 0.11 | 1    |

<br>
Critic score difference preference decision boundaries: [2,4,5,6,7,9,10,15]

| CS |    0 |    1 |    2 |    3 |    4 |   5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 0.33 | 2    | 0.12 | 0.25 | 8   | 0.25 | 0.12 | 1    | 0.17 | 0.5  | 0.5  | 0.25 | 0.17 | 1    | 0.12 | 2    | 7    | 1    |    9 |    9 | 8    | 3    | 1    | 7    |
|    1 | 3    | 1    | 6    | 0.2  | 1    | 9   | 1    | 0.25 | 2    | 0.5  | 1    | 1    | 1    | 0.5  | 4    | 0.2  | 6    | 8    | 5    |    9 |    9 | 9    | 7    | 4    | 8    |
|    2 | 0.5  | 0.17 | 1    | 0.12 | 0.17 | 8   | 0.17 | 0.12 | 0.33 | 0.12 | 0.25 | 0.2  | 0.17 | 0.12 | 1    | 0.12 | 1    | 5    | 1    |    8 |    9 | 6    | 1    | 1    | 5    |
|    3 | 8    | 5    | 8    | 1    | 4    | 9   | 4    | 1    | 7    | 2    | 6    | 6    | 4    | 2    | 8    | 1    | 9    | 9    | 8    |    9 |    9 | 9    | 9    | 8    | 9    |
|    4 | 4    | 1    | 6    | 0.25 | 1    | 9   | 1    | 0.33 | 2    | 0.5  | 2    | 1    | 1    | 0.5  | 5    | 0.25 | 7    | 9    | 6    |    9 |    9 | 9    | 8    | 5    | 9    |
|    5 | 0.12 | 0.11 | 0.12 | 0.11 | 0.11 | 1   | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.12 | 0.11 | 0.12 | 0.33 | 0.12 |    2 |    2 | 0.5  | 0.14 | 0.12 | 0.33 |
|    6 | 4    | 1    | 6    | 0.25 | 1    | 9   | 1    | 0.33 | 2    | 0.5  | 2    | 1    | 1    | 0.5  | 5    | 0.25 | 7    | 9    | 6    |    9 |    9 | 9    | 8    | 5    | 9    |
|    7 | 8    | 4    | 8    | 1    | 3    | 9   | 3    | 1    | 6    | 1    | 6    | 5    | 3    | 1    | 8    | 1    | 8    | 9    | 8    |    9 |    9 | 9    | 9    | 8    | 9    |
|    8 | 1    | 0.5  | 3    | 0.14 | 0.5  | 9   | 0.5  | 0.17 | 1    | 0.2  | 1    | 1    | 0.5  | 0.2  | 2    | 0.14 | 4    | 8    | 2    |    9 |    9 | 8    | 5    | 2    | 8    |
|    9 | 6    | 2    | 8    | 0.5  | 2    | 9   | 2    | 1    | 5    | 1    | 4    | 3    | 2    | 1    | 7    | 0.5  | 8    | 9    | 8    |    9 |    9 | 9    | 8    | 7    | 9    |
|   10 | 2    | 1    | 4    | 0.17 | 0.5  | 9   | 0.5  | 0.17 | 1    | 0.25 | 1    | 1    | 0.5  | 0.25 | 2    | 0.17 | 5    | 8    | 3    |    9 |    9 | 8    | 6    | 2    | 8    |
|   11 | 2    | 1    | 5    | 0.17 | 1    | 9   | 1    | 0.2  | 1    | 0.33 | 1    | 1    | 1    | 0.33 | 3    | 0.17 | 6    | 8    | 4    |    9 |    9 | 9    | 6    | 3    | 8    |
|   12 | 4    | 1    | 6    | 0.25 | 1    | 9   | 1    | 0.33 | 2    | 0.5  | 2    | 1    | 1    | 0.5  | 5    | 0.25 | 7    | 9    | 6    |    9 |    9 | 9    | 8    | 5    | 9    |
|   13 | 6    | 2    | 8    | 0.5  | 2    | 9   | 2    | 1    | 5    | 1    | 4    | 3    | 2    | 1    | 7    | 0.5  | 8    | 9    | 8    |    9 |    9 | 9    | 8    | 7    | 9    |
|   14 | 1    | 0.25 | 1    | 0.12 | 0.2  | 8   | 0.2  | 0.12 | 0.5  | 0.14 | 0.5  | 0.33 | 0.2  | 0.14 | 1    | 0.12 | 2    | 6    | 1    |    9 |    9 | 8    | 2    | 1    | 6    |
|   15 | 8    | 5    | 8    | 1    | 4    | 9   | 4    | 1    | 7    | 2    | 6    | 6    | 4    | 2    | 8    | 1    | 9    | 9    | 8    |    9 |    9 | 9    | 9    | 8    | 9    |
|   16 | 0.5  | 0.17 | 1    | 0.11 | 0.14 | 8   | 0.14 | 0.12 | 0.25 | 0.12 | 0.2  | 0.17 | 0.14 | 0.12 | 0.5  | 0.11 | 1    | 4    | 1    |    8 |    8 | 6    | 1    | 0.5  | 4    |
|   17 | 0.14 | 0.12 | 0.2  | 0.11 | 0.11 | 3   | 0.11 | 0.11 | 0.12 | 0.11 | 0.12 | 0.12 | 0.11 | 0.11 | 0.17 | 0.11 | 0.25 | 1    | 0.17 |    6 |    6 | 1    | 0.33 | 0.17 | 1    |
|   18 | 1    | 0.2  | 1    | 0.12 | 0.17 | 8   | 0.17 | 0.12 | 0.5  | 0.12 | 0.33 | 0.25 | 0.17 | 0.12 | 1    | 0.12 | 1    | 6    | 1    |    9 |    9 | 7    | 2    | 1    | 6    |
|   19 | 0.11 | 0.11 | 0.12 | 0.11 | 0.11 | 0.5 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.12 | 0.17 | 0.11 |    1 |    1 | 0.25 | 0.12 | 0.11 | 0.17 |
|   20 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.5 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.12 | 0.17 | 0.11 |    1 |    1 | 0.2  | 0.12 | 0.11 | 0.17 |
|   21 | 0.12 | 0.11 | 0.17 | 0.11 | 0.11 | 2   | 0.11 | 0.11 | 0.12 | 0.11 | 0.12 | 0.11 | 0.11 | 0.11 | 0.12 | 0.11 | 0.17 | 1    | 0.14 |    4 |    5 | 1    | 0.2  | 0.12 | 1    |
|   22 | 0.33 | 0.14 | 1    | 0.11 | 0.12 | 7   | 0.12 | 0.11 | 0.2  | 0.12 | 0.17 | 0.17 | 0.12 | 0.12 | 0.5  | 0.11 | 1    | 3    | 0.5  |    8 |    8 | 5    | 1    | 0.5  | 3    |
|   23 | 1    | 0.25 | 1    | 0.12 | 0.2  | 8   | 0.2  | 0.12 | 0.5  | 0.14 | 0.5  | 0.33 | 0.2  | 0.14 | 1    | 0.12 | 2    | 6    | 1    |    9 |    9 | 8    | 2    | 1    | 6    |
|   24 | 0.14 | 0.12 | 0.2  | 0.11 | 0.11 | 3   | 0.11 | 0.11 | 0.12 | 0.11 | 0.12 | 0.12 | 0.11 | 0.11 | 0.17 | 0.11 | 0.25 | 1    | 0.17 |    6 |    6 | 1    | 0.33 | 0.17 | 1    |

<br>
Genre score difference preference decision boundaries: [1,2,3,4,5,6,7,8]

| genres |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 1    | 2    | 3    | 4    | 1    | 1    | 4    | 2    | 2    | 6    | 3    | 1    | 5    | 2    | 1    | 5    | 6    |  7   | 6    | 3    | 1    |    9 |    8 | 3    |
|    1 | 1    | 1    | 2    | 3    | 4    | 1    | 1    | 4    | 2    | 2    | 6    | 3    | 1    | 5    | 2    | 1    | 5    | 6    |  7   | 6    | 3    | 1    |    9 |    8 | 3    |
|    2 | 0.5  | 0.5  | 1    | 1    | 2    | 1    | 1    | 2    | 1    | 1    | 4    | 1    | 1    | 3    | 1    | 0.5  | 3    | 4    |  5   | 4    | 1    | 0.5  |    7 |    6 | 1    |
|    3 | 0.33 | 0.33 | 1    | 1    | 1    | 0.5  | 0.5  | 1    | 1    | 1    | 3    | 1    | 0.5  | 2    | 1    | 0.33 | 2    | 3    |  4   | 3    | 1    | 0.33 |    6 |    5 | 1    |
|    4 | 0.25 | 0.25 | 0.5  | 1    | 1    | 0.33 | 0.33 | 1    | 0.5  | 0.5  | 2    | 1    | 0.33 | 1    | 0.5  | 0.25 | 1    | 2    |  3   | 2    | 1    | 0.25 |    5 |    4 | 1    |
|    5 | 1    | 1    | 1    | 2    | 3    | 1    | 1    | 3    | 1    | 1    | 5    | 2    | 1    | 4    | 1    | 1    | 4    | 5    |  6   | 5    | 2    | 1    |    8 |    7 | 2    |
|    6 | 1    | 1    | 1    | 2    | 3    | 1    | 1    | 3    | 1    | 1    | 5    | 2    | 1    | 4    | 1    | 1    | 4    | 5    |  6   | 5    | 2    | 1    |    8 |    7 | 2    |
|    7 | 0.25 | 0.25 | 0.5  | 1    | 1    | 0.33 | 0.33 | 1    | 0.5  | 0.5  | 2    | 1    | 0.33 | 1    | 0.5  | 0.25 | 1    | 2    |  3   | 2    | 1    | 0.25 |    5 |    4 | 1    |
|    8 | 0.5  | 0.5  | 1    | 1    | 2    | 1    | 1    | 2    | 1    | 1    | 4    | 1    | 1    | 3    | 1    | 0.5  | 3    | 4    |  5   | 4    | 1    | 0.5  |    7 |    6 | 1    |
|    9 | 0.5  | 0.5  | 1    | 1    | 2    | 1    | 1    | 2    | 1    | 1    | 4    | 1    | 1    | 3    | 1    | 0.5  | 3    | 4    |  5   | 4    | 1    | 0.5  |    7 |    6 | 1    |
|   10 | 0.17 | 0.17 | 0.25 | 0.33 | 0.5  | 0.2  | 0.2  | 0.5  | 0.25 | 0.25 | 1    | 0.33 | 0.2  | 1    | 0.25 | 0.17 | 1    | 1    |  1   | 1    | 0.33 | 0.17 |    3 |    2 | 0.33 |
|   11 | 0.33 | 0.33 | 1    | 1    | 1    | 0.5  | 0.5  | 1    | 1    | 1    | 3    | 1    | 0.5  | 2    | 1    | 0.33 | 2    | 3    |  4   | 3    | 1    | 0.33 |    6 |    5 | 1    |
|   12 | 1    | 1    | 1    | 2    | 3    | 1    | 1    | 3    | 1    | 1    | 5    | 2    | 1    | 4    | 1    | 1    | 4    | 5    |  6   | 5    | 2    | 1    |    8 |    7 | 2    |
|   13 | 0.2  | 0.2  | 0.33 | 0.5  | 1    | 0.25 | 0.25 | 1    | 0.33 | 0.33 | 1    | 0.5  | 0.25 | 1    | 0.33 | 0.2  | 1    | 1    |  2   | 1    | 0.5  | 0.2  |    4 |    3 | 0.5  |
|   14 | 0.5  | 0.5  | 1    | 1    | 2    | 1    | 1    | 2    | 1    | 1    | 4    | 1    | 1    | 3    | 1    | 0.5  | 3    | 4    |  5   | 4    | 1    | 0.5  |    7 |    6 | 1    |
|   15 | 1    | 1    | 2    | 3    | 4    | 1    | 1    | 4    | 2    | 2    | 6    | 3    | 1    | 5    | 2    | 1    | 5    | 6    |  7   | 6    | 3    | 1    |    9 |    8 | 3    |
|   16 | 0.2  | 0.2  | 0.33 | 0.5  | 1    | 0.25 | 0.25 | 1    | 0.33 | 0.33 | 1    | 0.5  | 0.25 | 1    | 0.33 | 0.2  | 1    | 1    |  2   | 1    | 0.5  | 0.2  |    4 |    3 | 0.5  |
|   17 | 0.17 | 0.17 | 0.25 | 0.33 | 0.5  | 0.2  | 0.2  | 0.5  | 0.25 | 0.25 | 1    | 0.33 | 0.2  | 1    | 0.25 | 0.17 | 1    | 1    |  1   | 1    | 0.33 | 0.17 |    3 |    2 | 0.33 |
|   18 | 0.14 | 0.14 | 0.2  | 0.25 | 0.33 | 0.17 | 0.17 | 0.33 | 0.2  | 0.2  | 1    | 0.25 | 0.17 | 0.5  | 0.2  | 0.14 | 0.5  | 1    |  1   | 1    | 0.25 | 0.14 |    2 |    1 | 0.25 |
|   19 | 0.17 | 0.17 | 0.25 | 0.33 | 0.5  | 0.2  | 0.2  | 0.5  | 0.25 | 0.25 | 1    | 0.33 | 0.2  | 1    | 0.25 | 0.17 | 1    | 1    |  1   | 1    | 0.33 | 0.17 |    3 |    2 | 0.33 |
|   20 | 0.33 | 0.33 | 1    | 1    | 1    | 0.5  | 0.5  | 1    | 1    | 1    | 3    | 1    | 0.5  | 2    | 1    | 0.33 | 2    | 3    |  4   | 3    | 1    | 0.33 |    6 |    5 | 1    |
|   21 | 1    | 1    | 2    | 3    | 4    | 1    | 1    | 4    | 2    | 2    | 6    | 3    | 1    | 5    | 2    | 1    | 5    | 6    |  7   | 6    | 3    | 1    |    9 |    8 | 3    |
|   22 | 0.11 | 0.11 | 0.14 | 0.17 | 0.2  | 0.12 | 0.12 | 0.2  | 0.14 | 0.14 | 0.33 | 0.17 | 0.12 | 0.25 | 0.14 | 0.11 | 0.25 | 0.33 |  0.5 | 0.33 | 0.17 | 0.11 |    1 |    1 | 0.17 |
|   23 | 0.12 | 0.12 | 0.17 | 0.2  | 0.25 | 0.14 | 0.14 | 0.25 | 0.17 | 0.17 | 0.5  | 0.2  | 0.14 | 0.33 | 0.17 | 0.12 | 0.33 | 0.5  |  1   | 0.5  | 0.2  | 0.12 |    1 |    1 | 0.2  |
|   24 | 0.33 | 0.33 | 1    | 1    | 1    | 0.5  | 0.5  | 1    | 1    | 1    | 3    | 1    | 0.5  | 2    | 1    | 0.33 | 2    | 3    |  4   | 3    | 1    | 0.33 |    6 |    5 | 1    |

<br>
Length difference preference decision boundaries: [7, 12, 16, 25, 33, 40, 45, 60]

| length |    0 |    1 |    2 |    3 |    4 |    5 |    6 |   7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 1    | 0.12 | 0.11 | 2    | 1    | 1    | 5   | 4    | 1    | 1    | 5    |  5   |    6 | 3    | 0.11 | 4    | 2    | 0.25 |    6 |    6 | 2    | 4    | 0.17 |    6 |
|    1 | 1    | 1    | 0.12 | 0.12 | 4    | 2    | 1    | 6   | 4    | 1    | 1    | 5    |  5   |    7 | 4    | 0.11 | 5    | 3    | 0.33 |    6 |    7 | 3    | 5    | 0.2  |    7 |
|    2 | 8    | 8    | 1    | 1    | 9    | 9    | 8    | 9   | 9    | 8    | 8    | 9    |  9   |    9 | 9    | 0.5  | 9    | 9    | 6    |    9 |    9 | 9    | 9    | 4    |    9 |
|    3 | 9    | 8    | 1    | 1    | 9    | 9    | 9    | 9   | 9    | 8    | 9    | 9    |  9   |    9 | 9    | 1    | 9    | 9    | 7    |    9 |    9 | 9    | 9    | 5    |    9 |
|    4 | 0.5  | 0.25 | 0.11 | 0.11 | 1    | 1    | 0.5  | 4   | 1    | 0.25 | 0.5  | 3    |  3   |    5 | 1    | 0.11 | 2    | 1    | 0.2  |    4 |    4 | 1    | 2    | 0.12 |    5 |
|    5 | 1    | 0.5  | 0.11 | 0.11 | 1    | 1    | 1    | 4   | 2    | 0.5  | 1    | 4    |  4   |    5 | 2    | 0.11 | 3    | 1    | 0.25 |    5 |    5 | 1    | 3    | 0.14 |    5 |
|    6 | 1    | 1    | 0.12 | 0.11 | 2    | 1    | 1    | 5   | 4    | 1    | 1    | 4    |  5   |    6 | 3    | 0.11 | 4    | 1    | 0.25 |    5 |    6 | 1    | 4    | 0.17 |    6 |
|    7 | 0.2  | 0.17 | 0.11 | 0.11 | 0.25 | 0.25 | 0.2  | 1   | 0.5  | 0.17 | 0.2  | 1    |  1   |    2 | 0.33 | 0.11 | 0.5  | 0.25 | 0.12 |    1 |    1 | 0.25 | 0.5  | 0.11 |    2 |
|    8 | 0.25 | 0.25 | 0.11 | 0.11 | 1    | 0.5  | 0.25 | 2   | 1    | 0.25 | 0.25 | 1    |  2   |    4 | 1    | 0.11 | 1    | 0.5  | 0.17 |    3 |    4 | 0.5  | 1    | 0.12 |    4 |
|    9 | 1    | 1    | 0.12 | 0.12 | 4    | 2    | 1    | 6   | 4    | 1    | 1    | 5    |  5   |    7 | 4    | 0.11 | 5    | 3    | 0.33 |    6 |    7 | 3    | 5    | 0.2  |    7 |
|   10 | 1    | 1    | 0.12 | 0.11 | 2    | 1    | 1    | 5   | 4    | 1    | 1    | 4    |  5   |    6 | 3    | 0.11 | 4    | 1    | 0.25 |    5 |    6 | 2    | 4    | 0.17 |    6 |
|   11 | 0.2  | 0.2  | 0.11 | 0.11 | 0.33 | 0.25 | 0.25 | 1   | 1    | 0.2  | 0.25 | 1    |  1   |    3 | 0.5  | 0.11 | 1    | 0.25 | 0.14 |    2 |    2 | 0.25 | 1    | 0.12 |    3 |
|   12 | 0.2  | 0.2  | 0.11 | 0.11 | 0.33 | 0.25 | 0.2  | 1   | 0.5  | 0.2  | 0.2  | 1    |  1   |    2 | 0.5  | 0.11 | 1    | 0.25 | 0.12 |    1 |    2 | 0.25 | 1    | 0.11 |    2 |
|   13 | 0.17 | 0.14 | 0.11 | 0.11 | 0.2  | 0.2  | 0.17 | 0.5 | 0.25 | 0.14 | 0.17 | 0.33 |  0.5 |    1 | 0.25 | 0.11 | 0.25 | 0.2  | 0.12 |    1 |    1 | 0.2  | 0.25 | 0.11 |    1 |
|   14 | 0.33 | 0.25 | 0.11 | 0.11 | 1    | 0.5  | 0.33 | 3   | 1    | 0.25 | 0.33 | 2    |  2   |    4 | 1    | 0.11 | 1    | 0.5  | 0.17 |    4 |    4 | 1    | 1    | 0.12 |    4 |
|   15 | 9    | 9    | 2    | 1    | 9    | 9    | 9    | 9   | 9    | 9    | 9    | 9    |  9   |    9 | 9    | 1    | 9    | 9    | 8    |    9 |    9 | 9    | 9    | 6    |    9 |
|   16 | 0.25 | 0.2  | 0.11 | 0.11 | 0.5  | 0.33 | 0.25 | 2   | 1    | 0.2  | 0.25 | 1    |  1   |    4 | 1    | 0.11 | 1    | 0.33 | 0.17 |    2 |    3 | 0.33 | 1    | 0.12 |    4 |
|   17 | 0.5  | 0.33 | 0.11 | 0.11 | 1    | 1    | 1    | 4   | 2    | 0.33 | 1    | 4    |  4   |    5 | 2    | 0.11 | 3    | 1    | 0.2  |    5 |    5 | 1    | 3    | 0.14 |    5 |
|   18 | 4    | 3    | 0.17 | 0.14 | 5    | 4    | 4    | 8   | 6    | 3    | 4    | 7    |  8   |    8 | 6    | 0.12 | 6    | 5    | 1    |    8 |    8 | 5    | 6    | 0.33 |    8 |
|   19 | 0.17 | 0.17 | 0.11 | 0.11 | 0.25 | 0.2  | 0.2  | 1   | 0.33 | 0.17 | 0.2  | 0.5  |  1   |    1 | 0.25 | 0.11 | 0.5  | 0.2  | 0.12 |    1 |    1 | 0.25 | 0.33 | 0.11 |    1 |
|   20 | 0.17 | 0.14 | 0.11 | 0.11 | 0.25 | 0.2  | 0.17 | 1   | 0.25 | 0.14 | 0.17 | 0.5  |  0.5 |    1 | 0.25 | 0.11 | 0.33 | 0.2  | 0.12 |    1 |    1 | 0.2  | 0.33 | 0.11 |    1 |
|   21 | 0.5  | 0.33 | 0.11 | 0.11 | 1    | 1    | 1    | 4   | 2    | 0.33 | 0.5  | 4    |  4   |    5 | 1    | 0.11 | 3    | 1    | 0.2  |    4 |    5 | 1    | 2    | 0.14 |    5 |
|   22 | 0.25 | 0.2  | 0.11 | 0.11 | 0.5  | 0.33 | 0.25 | 2   | 1    | 0.2  | 0.25 | 1    |  1   |    4 | 1    | 0.11 | 1    | 0.33 | 0.17 |    3 |    3 | 0.5  | 1    | 0.12 |    4 |
|   23 | 6    | 5    | 0.25 | 0.2  | 8    | 7    | 6    | 9   | 8    | 5    | 6    | 8    |  9   |    9 | 8    | 0.17 | 8    | 7    | 3    |    9 |    9 | 7    | 8    | 1    |    9 |
|   24 | 0.17 | 0.14 | 0.11 | 0.11 | 0.2  | 0.2  | 0.17 | 0.5 | 0.25 | 0.14 | 0.17 | 0.33 |  0.5 |    1 | 0.25 | 0.11 | 0.25 | 0.2  | 0.12 |    1 |    1 | 0.2  | 0.25 | 0.11 |    1 |

<br>
Number of achievements difference preference decision boundaries: [5, 10, 15, 23, 35, 50, 65, 80]

| NoA |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 1    | 0.12 | 0.33 | 1    | 0.14 | 0.25 | 0.5  | 0.11 | 0.5  |    5 | 0.17 | 1    |    5 | 1    | 0.2  | 3    | 0.17 | 0.33 |  4   | 3    | 0.25 | 0.5  | 0.2  | 3    |
|    1 | 1    | 1    | 0.12 | 0.33 | 1    | 0.17 | 0.25 | 0.5  | 0.11 | 0.5  |    5 | 0.17 | 2    |    5 | 1    | 0.2  | 4    | 0.2  | 0.33 |  4   | 4    | 0.33 | 0.5  | 0.25 | 4    |
|    2 | 8    | 8    | 1    | 7    | 8    | 4    | 7    | 7    | 0.11 | 8    |    9 | 5    | 8    |    9 | 8    | 6    | 9    | 6    | 7    |  9   | 9    | 7    | 7    | 6    | 9    |
|    3 | 3    | 3    | 0.14 | 1    | 3    | 0.17 | 0.5  | 1    | 0.11 | 1    |    6 | 0.2  | 4    |    6 | 2    | 0.25 | 5    | 0.2  | 1    |  5   | 5    | 1    | 1    | 0.33 | 5    |
|    4 | 1    | 1    | 0.12 | 0.33 | 1    | 0.17 | 0.25 | 0.5  | 0.11 | 0.5  |    5 | 0.17 | 2    |    5 | 1    | 0.2  | 4    | 0.2  | 0.33 |  4   | 4    | 0.33 | 0.5  | 0.25 | 4    |
|    5 | 7    | 6    | 0.25 | 6    | 6    | 1    | 5    | 6    | 0.11 | 6    |    8 | 3    | 7    |    8 | 6    | 4    | 7    | 3    | 6    |  8   | 8    | 5    | 6    | 5    | 7    |
|    6 | 4    | 4    | 0.14 | 2    | 4    | 0.2  | 1    | 3    | 0.11 | 3    |    6 | 0.25 | 5    |    6 | 4    | 0.33 | 6    | 0.33 | 2    |  6   | 6    | 2    | 3    | 1    | 6    |
|    7 | 2    | 2    | 0.14 | 1    | 2    | 0.17 | 0.33 | 1    | 0.11 | 1    |    5 | 0.2  | 3    |    6 | 2    | 0.2  | 5    | 0.2  | 1    |  5   | 5    | 0.5  | 1    | 0.33 | 5    |
|    8 | 9    | 9    | 9    | 9    | 9    | 9    | 9    | 9    | 1    | 9    |    9 | 9    | 9    |    9 | 9    | 9    | 9    | 9    | 9    |  9   | 9    | 9    | 9    | 9    | 9    |
|    9 | 2    | 2    | 0.12 | 1    | 2    | 0.17 | 0.33 | 1    | 0.11 | 1    |    5 | 0.2  | 3    |    5 | 1    | 0.2  | 4    | 0.2  | 1    |  5   | 4    | 0.5  | 1    | 0.25 | 4    |
|   10 | 0.2  | 0.2  | 0.11 | 0.17 | 0.2  | 0.12 | 0.17 | 0.2  | 0.11 | 0.2  |    1 | 0.14 | 0.25 |    1 | 0.2  | 0.14 | 0.5  | 0.14 | 0.17 |  1   | 0.5  | 0.17 | 0.2  | 0.17 | 0.5  |
|   11 | 6    | 6    | 0.2  | 5    | 6    | 0.33 | 4    | 5    | 0.11 | 5    |    7 | 1    | 6    |    8 | 5    | 1    | 7    | 1    | 5    |  7   | 7    | 4    | 5    | 3    | 7    |
|   12 | 1    | 0.5  | 0.12 | 0.25 | 0.5  | 0.14 | 0.2  | 0.33 | 0.11 | 0.33 |    4 | 0.17 | 1    |    4 | 0.5  | 0.17 | 2    | 0.17 | 0.25 |  3   | 3    | 0.25 | 0.33 | 0.2  | 2    |
|   13 | 0.2  | 0.2  | 0.11 | 0.17 | 0.2  | 0.12 | 0.17 | 0.17 | 0.11 | 0.2  |    1 | 0.12 | 0.25 |    1 | 0.2  | 0.14 | 0.33 | 0.14 | 0.17 |  0.5 | 0.33 | 0.17 | 0.17 | 0.14 | 0.33 |
|   14 | 1    | 1    | 0.12 | 0.5  | 1    | 0.17 | 0.25 | 0.5  | 0.11 | 1    |    5 | 0.2  | 2    |    5 | 1    | 0.2  | 4    | 0.2  | 0.5  |  4   | 4    | 0.33 | 0.5  | 0.25 | 4    |
|   15 | 5    | 5    | 0.17 | 4    | 5    | 0.25 | 3    | 5    | 0.11 | 5    |    7 | 1    | 6    |    7 | 5    | 1    | 6    | 1    | 4    |  7   | 6    | 4    | 5    | 2    | 6    |
|   16 | 0.33 | 0.25 | 0.11 | 0.2  | 0.25 | 0.14 | 0.17 | 0.2  | 0.11 | 0.25 |    2 | 0.14 | 0.5  |    3 | 0.25 | 0.17 | 1    | 0.14 | 0.2  |  1   | 1    | 0.2  | 0.2  | 0.17 | 1    |
|   17 | 6    | 5    | 0.17 | 5    | 5    | 0.33 | 3    | 5    | 0.11 | 5    |    7 | 1    | 6    |    7 | 5    | 1    | 7    | 1    | 5    |  7   | 7    | 4    | 5    | 3    | 7    |
|   18 | 3    | 3    | 0.14 | 1    | 3    | 0.17 | 0.5  | 1    | 0.11 | 1    |    6 | 0.2  | 4    |    6 | 2    | 0.25 | 5    | 0.2  | 1    |  5   | 5    | 1    | 1    | 0.33 | 5    |
|   19 | 0.25 | 0.25 | 0.11 | 0.2  | 0.25 | 0.12 | 0.17 | 0.2  | 0.11 | 0.2  |    1 | 0.14 | 0.33 |    2 | 0.25 | 0.14 | 1    | 0.14 | 0.2  |  1   | 1    | 0.2  | 0.2  | 0.17 | 1    |
|   20 | 0.33 | 0.25 | 0.11 | 0.2  | 0.25 | 0.12 | 0.17 | 0.2  | 0.11 | 0.25 |    2 | 0.14 | 0.33 |    3 | 0.25 | 0.17 | 1    | 0.14 | 0.2  |  1   | 1    | 0.2  | 0.2  | 0.17 | 1    |
|   21 | 4    | 3    | 0.14 | 1    | 3    | 0.2  | 0.5  | 2    | 0.11 | 2    |    6 | 0.25 | 4    |    6 | 3    | 0.25 | 5    | 0.25 | 1    |  5   | 5    | 1    | 2    | 0.5  | 5    |
|   22 | 2    | 2    | 0.14 | 1    | 2    | 0.17 | 0.33 | 1    | 0.11 | 1    |    5 | 0.2  | 3    |    6 | 2    | 0.2  | 5    | 0.2  | 1    |  5   | 5    | 0.5  | 1    | 0.33 | 5    |
|   23 | 5    | 4    | 0.17 | 3    | 4    | 0.2  | 1    | 3    | 0.11 | 4    |    6 | 0.33 | 5    |    7 | 4    | 0.5  | 6    | 0.33 | 3    |  6   | 6    | 2    | 3    | 1    | 6    |
|   24 | 0.33 | 0.25 | 0.11 | 0.2  | 0.25 | 0.14 | 0.17 | 0.2  | 0.11 | 0.25 |    2 | 0.14 | 0.5  |    3 | 0.25 | 0.17 | 1    | 0.14 | 0.2  |  1   | 1    | 0.2  | 0.2  | 0.17 | 1    |

<br>
Price difference preference decision boundaries: [5, 12, 20, 35, 50, 80, 120, 150]

| price |    0 |   1 |    2 |   3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|----:|-----:|----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 5   | 0.14 |   7 | 0.17 | 1    | 0.14 | 0.14 | 0.12 | 0.25 | 0.5  | 0.14 | 0.2  | 0.17 | 0.17 | 0.17 | 0.25 | 0.17 | 0.14 | 3    | 0.25 | 0.25 | 0.17 | 0.12 | 0.14 |
|    1 | 0.2  | 1   | 0.11 |   5 | 0.14 | 0.2  | 0.12 | 0.11 | 0.11 | 0.14 | 0.17 | 0.11 | 0.14 | 0.12 | 0.12 | 0.14 | 0.17 | 0.14 | 0.12 | 0.25 | 0.17 | 0.17 | 0.14 | 0.11 | 0.12 |
|    2 | 7    | 9   | 1    |   9 | 5    | 7    | 4    | 1    | 0.25 | 6    | 7    | 1    | 6    | 4    | 4    | 5    | 6    | 5    | 4    | 8    | 7    | 6    | 4    | 0.33 | 4    |
|    3 | 0.14 | 0.2 | 0.11 |   1 | 0.11 | 0.14 | 0.11 | 0.11 | 0.11 | 0.12 | 0.14 | 0.11 | 0.12 | 0.11 | 0.11 | 0.11 | 0.12 | 0.11 | 0.11 | 0.14 | 0.12 | 0.12 | 0.11 | 0.11 | 0.11 |
|    4 | 6    | 7   | 0.2  |   9 | 1    | 6    | 0.25 | 0.2  | 0.17 | 4    | 5    | 0.2  | 3    | 0.33 | 0.33 | 1    | 4    | 1    | 0.25 | 6    | 5    | 4    | 0.5  | 0.17 | 0.25 |
|    5 | 1    | 5   | 0.14 |   7 | 0.17 | 1    | 0.14 | 0.14 | 0.12 | 0.2  | 0.33 | 0.14 | 0.2  | 0.14 | 0.14 | 0.17 | 0.25 | 0.17 | 0.14 | 3    | 0.25 | 0.25 | 0.17 | 0.12 | 0.14 |
|    6 | 7    | 8   | 0.25 |   9 | 4    | 7    | 1    | 0.25 | 0.2  | 5    | 6    | 0.25 | 5    | 1    | 1    | 4    | 6    | 4    | 1    | 7    | 6    | 6    | 2    | 0.2  | 1    |
|    7 | 7    | 9   | 1    |   9 | 5    | 7    | 4    | 1    | 0.25 | 6    | 7    | 1    | 6    | 4    | 4    | 5    | 6    | 5    | 4    | 8    | 7    | 6    | 4    | 0.33 | 4    |
|    8 | 8    | 9   | 4    |   9 | 6    | 8    | 5    | 4    | 1    | 7    | 7    | 4    | 7    | 6    | 6    | 6    | 7    | 6    | 5    | 8    | 7    | 7    | 6    | 2    | 5    |
|    9 | 4    | 7   | 0.17 |   8 | 0.25 | 5    | 0.2  | 0.17 | 0.14 | 1    | 4    | 0.17 | 0.5  | 0.2  | 0.2  | 0.25 | 1    | 0.25 | 0.2  | 6    | 3    | 1    | 0.25 | 0.14 | 0.2  |
|   10 | 2    | 6   | 0.14 |   7 | 0.2  | 3    | 0.17 | 0.14 | 0.14 | 0.25 | 1    | 0.14 | 0.25 | 0.17 | 0.17 | 0.2  | 0.33 | 0.2  | 0.17 | 4    | 0.5  | 0.33 | 0.17 | 0.14 | 0.17 |
|   11 | 7    | 9   | 1    |   9 | 5    | 7    | 4    | 1    | 0.25 | 6    | 7    | 1    | 6    | 4    | 4    | 5    | 6    | 5    | 4    | 8    | 7    | 6    | 4    | 0.33 | 4    |
|   12 | 5    | 7   | 0.17 |   8 | 0.33 | 5    | 0.2  | 0.17 | 0.14 | 2    | 4    | 0.17 | 1    | 0.2  | 0.2  | 0.33 | 2    | 0.33 | 0.2  | 6    | 4    | 2    | 0.25 | 0.17 | 0.2  |
|   13 | 6    | 8   | 0.25 |   9 | 3    | 7    | 1    | 0.25 | 0.17 | 5    | 6    | 0.25 | 5    | 1    | 1    | 3    | 5    | 3    | 1    | 7    | 6    | 5    | 2    | 0.2  | 1    |
|   14 | 6    | 8   | 0.25 |   9 | 3    | 7    | 1    | 0.25 | 0.17 | 5    | 6    | 0.25 | 5    | 1    | 1    | 3    | 5    | 3    | 1    | 7    | 6    | 5    | 2    | 0.2  | 1    |
|   15 | 6    | 7   | 0.2  |   9 | 1    | 6    | 0.25 | 0.2  | 0.17 | 4    | 5    | 0.2  | 3    | 0.33 | 0.33 | 1    | 4    | 1    | 0.25 | 6    | 5    | 4    | 0.5  | 0.17 | 0.25 |
|   16 | 4    | 6   | 0.17 |   8 | 0.25 | 4    | 0.17 | 0.17 | 0.14 | 1    | 3    | 0.17 | 0.5  | 0.2  | 0.2  | 0.25 | 1    | 0.25 | 0.17 | 5    | 2    | 1    | 0.2  | 0.14 | 0.17 |
|   17 | 6    | 7   | 0.2  |   9 | 1    | 6    | 0.25 | 0.2  | 0.17 | 4    | 5    | 0.2  | 3    | 0.33 | 0.33 | 1    | 4    | 1    | 0.25 | 6    | 5    | 4    | 0.5  | 0.17 | 0.25 |
|   18 | 7    | 8   | 0.25 |   9 | 4    | 7    | 1    | 0.25 | 0.2  | 5    | 6    | 0.25 | 5    | 1    | 1    | 4    | 6    | 4    | 1    | 7    | 6    | 6    | 2    | 0.2  | 1    |
|   19 | 0.33 | 4   | 0.12 |   7 | 0.17 | 0.33 | 0.14 | 0.12 | 0.12 | 0.17 | 0.25 | 0.12 | 0.17 | 0.14 | 0.14 | 0.17 | 0.2  | 0.17 | 0.14 | 1    | 0.2  | 0.2  | 0.14 | 0.12 | 0.14 |
|   20 | 4    | 6   | 0.14 |   8 | 0.2  | 4    | 0.17 | 0.14 | 0.14 | 0.33 | 2    | 0.14 | 0.25 | 0.17 | 0.17 | 0.2  | 0.5  | 0.2  | 0.17 | 5    | 1    | 0.5  | 0.2  | 0.14 | 0.17 |
|   21 | 4    | 6   | 0.17 |   8 | 0.25 | 4    | 0.17 | 0.17 | 0.14 | 1    | 3    | 0.17 | 0.5  | 0.2  | 0.2  | 0.25 | 1    | 0.25 | 0.17 | 5    | 2    | 1    | 0.2  | 0.14 | 0.17 |
|   22 | 6    | 7   | 0.25 |   9 | 2    | 6    | 0.5  | 0.25 | 0.17 | 4    | 6    | 0.25 | 4    | 0.5  | 0.5  | 2    | 5    | 2    | 0.5  | 7    | 5    | 5    | 1    | 0.17 | 0.5  |
|   23 | 8    | 9   | 3    |   9 | 6    | 8    | 5    | 3    | 0.5  | 7    | 7    | 3    | 6    | 5    | 5    | 6    | 7    | 6    | 5    | 8    | 7    | 7    | 6    | 1    | 5    |
|   24 | 7    | 8   | 0.25 |   9 | 4    | 7    | 1    | 0.25 | 0.2  | 5    | 6    | 0.25 | 5    | 1    | 1    | 4    | 6    | 4    | 1    | 7    | 6    | 6    | 2    | 0.2  | 1    |

<br>

| quality      |   user_score |   critic_score |   genres |
|:-------------|-------------:|---------------:|---------:|
| user_score   |     1        |        4       |     3    |
| critic_score |     0.25     |        1       |     0.75 |
| genres       |     0.333333 |        1.33333 |     1    |

<br>

| quantity            |    price |   length |   num_of_achievements |
|:--------------------|---------:|---------:|----------------------:|
| price               | 1        | 5        |                   7   |
| length              | 0.2      | 1        |                   1.4 |
| num_of_achievements | 0.142857 | 0.714286 |                   1   |

<br>

| goal     |   quality |   quantity |
|:---------|----------:|-----------:|
| quality  |      1    |          4 |
| quantity |      0.25 |          1 |

#### Pairwise comparisons

In [22]:
boundaries_user_score = [2,4,5,6,7,9,10,15]
comparisons_user_score = get_pairwise_comparisons(data, "user_score", boundaries_user_score)
print(comparisons_user_score.to_markdown())

|   id |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|    1 | 5    | 1    | 6    | 1    | 5    | 9    | 1    | 1    | 5    | 1    | 2    | 5    | 1    | 5    | 8    | 2    | 6    | 6    | 7    |    9 | 8    | 9    | 9    | 8    | 9    |
|    2 | 1    | 0.17 | 1    | 0.17 | 1    | 9    | 0.14 | 0.17 | 1    | 0.2  | 0.33 | 1    | 0.17 | 1    | 4    | 0.33 | 1    | 1    | 1    |    9 | 4    | 9    | 7    | 2    | 9    |
|    3 | 4    | 1    | 6    | 1    | 4    | 9    | 1    | 1    | 4    | 1    | 2

In [23]:
boundaries_critic_score = [2,4,5,6,7,9,10,15]
comparisons_critic_score = get_pairwise_comparisons(data, "critic_score", boundaries_critic_score)
print(comparisons_critic_score.to_markdown())

|   id |    0 |    1 |    2 |    3 |    4 |   5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 0.33 | 2    | 0.12 | 0.25 | 8   | 0.25 | 0.12 | 1    | 0.17 | 0.5  | 0.5  | 0.25 | 0.17 | 1    | 0.12 | 2    | 7    | 1    |    9 |    9 | 8    | 3    | 1    | 7    |
|    1 | 3    | 1    | 6    | 0.2  | 1    | 9   | 1    | 0.25 | 2    | 0.5  | 1    | 1    | 1    | 0.5  | 4    | 0.2  | 6    | 8    | 5    |    9 |    9 | 9    | 7    | 4    | 8    |
|    2 | 0.5  | 0.17 | 1    | 0.12 | 0.17 | 8   | 0.17 | 0.12 | 0.33 | 0.12 | 0.25 | 0.2  | 0.17 | 0.12 | 1    | 0.12 | 1    | 5    | 1    |    8 |    9 | 6    | 1    | 1    | 5    |
|    3 | 8    | 5    | 8    | 1    | 4    | 9   | 4    | 1    | 7    | 2    | 6    | 

In [24]:
boundaries_genres = [1,2,3,4,5,6,7,8]
comparisons_genres_score = get_pairwise_comparisons(data, "genres", boundaries_genres)
print(comparisons_genres_score.to_markdown())

|   id |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 1    | 2    | 3    | 4    | 1    | 1    | 4    | 2    | 2    | 6    | 3    | 1    | 5    | 2    | 1    | 5    | 6    |  7   | 6    | 3    | 1    |    9 |    8 | 3    |
|    1 | 1    | 1    | 2    | 3    | 4    | 1    | 1    | 4    | 2    | 2    | 6    | 3    | 1    | 5    | 2    | 1    | 5    | 6    |  7   | 6    | 3    | 1    |    9 |    8 | 3    |
|    2 | 0.5  | 0.5  | 1    | 1    | 2    | 1    | 1    | 2    | 1    | 1    | 4    | 1    | 1    | 3    | 1    | 0.5  | 3    | 4    |  5   | 4    | 1    | 0.5  |    7 |    6 | 1    |
|    3 | 0.33 | 0.33 | 1    | 1    | 1    | 0.5  | 0.5  | 1    | 1    | 1    | 3

In [25]:
boundaries_length = [7, 12, 16, 25, 33, 40, 45, 60]
comparisons_length = get_pairwise_comparisons(data, "length", boundaries_length)
print(comparisons_length.to_markdown())

|   id |    0 |    1 |    2 |    3 |    4 |    5 |    6 |   7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 1    | 0.12 | 0.11 | 2    | 1    | 1    | 5   | 4    | 1    | 1    | 5    |  5   |    6 | 3    | 0.11 | 4    | 2    | 0.25 |    6 |    6 | 2    | 4    | 0.17 |    6 |
|    1 | 1    | 1    | 0.12 | 0.12 | 4    | 2    | 1    | 6   | 4    | 1    | 1    | 5    |  5   |    7 | 4    | 0.11 | 5    | 3    | 0.33 |    6 |    7 | 3    | 5    | 0.2  |    7 |
|    2 | 8    | 8    | 1    | 1    | 9    | 9    | 8    | 9   | 9    | 8    | 8    | 9    |  9   |    9 | 9    | 0.5  | 9    | 9    | 6    |    9 |    9 | 9    | 9    | 4    |    9 |
|    3 | 9    | 8    | 1    | 1    | 9    | 9    | 9    | 9   | 9    | 8    | 9    | 

In [26]:
boundaries_num_of_achievements = [5, 10, 15, 23, 35, 50, 65, 80]
comparisons_num_of_achievements = get_pairwise_comparisons(data, "num_of_achievements", boundaries_num_of_achievements)
print(comparisons_num_of_achievements.to_markdown())

|   id |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 1    | 0.12 | 0.33 | 1    | 0.14 | 0.25 | 0.5  | 0.11 | 0.5  |    5 | 0.17 | 1    |    5 | 1    | 0.2  | 3    | 0.17 | 0.33 |  4   | 3    | 0.25 | 0.5  | 0.2  | 3    |
|    1 | 1    | 1    | 0.12 | 0.33 | 1    | 0.17 | 0.25 | 0.5  | 0.11 | 0.5  |    5 | 0.17 | 2    |    5 | 1    | 0.2  | 4    | 0.2  | 0.33 |  4   | 4    | 0.33 | 0.5  | 0.25 | 4    |
|    2 | 8    | 8    | 1    | 7    | 8    | 4    | 7    | 7    | 0.11 | 8    |    9 | 5    | 8    |    9 | 8    | 6    | 9    | 6    | 7    |  9   | 9    | 7    | 7    | 6    | 9    |
|    3 | 3    | 3    | 0.14 | 1    | 3    | 0.17 | 0.5  | 1    | 0.11 | 1    |  

In [27]:
boundaries_price = [5, 12, 20, 35, 50, 80, 120, 150]
comparisons_price = get_pairwise_comparisons(data, "price", boundaries_price, cost=True)
print(comparisons_price.to_markdown())

|   id |    0 |   1 |    2 |   3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|----:|-----:|----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 5   | 0.14 |   7 | 0.17 | 1    | 0.14 | 0.14 | 0.12 | 0.25 | 0.5  | 0.14 | 0.2  | 0.17 | 0.17 | 0.17 | 0.25 | 0.17 | 0.14 | 3    | 0.25 | 0.25 | 0.17 | 0.12 | 0.14 |
|    1 | 0.2  | 1   | 0.11 |   5 | 0.14 | 0.2  | 0.12 | 0.11 | 0.11 | 0.14 | 0.17 | 0.11 | 0.14 | 0.12 | 0.12 | 0.14 | 0.17 | 0.14 | 0.12 | 0.25 | 0.17 | 0.17 | 0.14 | 0.11 | 0.12 |
|    2 | 7    | 9   | 1    |   9 | 5    | 7    | 4    | 1    | 0.25 | 6    | 7    | 1    | 6    | 4    | 4    | 5    | 6    | 5    | 4    | 8    | 7    | 6    | 4    | 0.33 | 4    |
|    3 | 0.14 | 0.2 | 0.11 |   1 | 0.11 | 0.14 | 0.11 | 0.11 | 0.11 | 0.12 | 0.14 | 0.11 |

In [30]:
quality_criteria_names = ["user_score", "critic_score", "genres"]
quality_criteria_proportions = [
    [1,   4,   3  ],
    [1/4, 1,   3/4],
    [1/3, 4/3, 1  ]
]
comparisons_quality_criteria = pd.DataFrame(quality_criteria_proportions, columns=quality_criteria_names, index=quality_criteria_names)
print(comparisons_quality_criteria.to_markdown())

|              |   user_score |   critic_score |   genres |
|:-------------|-------------:|---------------:|---------:|
| user_score   |     1        |        4       |     3    |
| critic_score |     0.25     |        1       |     0.75 |
| genres       |     0.333333 |        1.33333 |     1    |


In [32]:
quantity_criteria_names = ["price", "length", "num_of_achievements"]
quantity_criteria_proportions = [
    [1,   5,   7  ],
    [1/5, 1,   7/5],
    [1/7, 5/7, 1  ]
]
comparisons_quantity_criteria = pd.DataFrame(quantity_criteria_proportions, columns=quantity_criteria_names, index=quantity_criteria_names)
print(comparisons_quantity_criteria.to_markdown())

|                     |    price |   length |   num_of_achievements |
|:--------------------|---------:|---------:|----------------------:|
| price               | 1        | 5        |                   7   |
| length              | 0.2      | 1        |                   1.4 |
| num_of_achievements | 0.142857 | 0.714286 |                   1   |


In [35]:
criteria_names = ["quality", "quantity"]
criteria_proportions = [
    [1,   4],
    [1/4, 1]
]
comparisons_criteria = pd.DataFrame(criteria_proportions, columns=criteria_names, index=criteria_names)
print(comparisons_criteria.to_markdown())

|          |   quality |   quantity |
|:---------|----------:|-----------:|
| quality  |      1    |          4 |
| quantity |      0.25 |          1 |


## TESTS

In [ ]:
quantity_criteria_names = ["price", "length", "num_of_achievements"]
quantity_criteria_proportions = [
    [1,   5,   7  ],
    [1/5, 1,   7/5],
    [1/7, 5/7, 1  ]
]
comparisons_quantity_criteria = pd.DataFrame(quantity_criteria_proportions, columns=quantity_criteria_names, index=quantity_criteria_names)
print(comparisons_quantity_criteria.to_markdown())